#### Libraries

In [1]:
import pandas as pd
import requests
import calendar
from datetime import datetime, timedelta

#### Calling the API and create Dataframes with Pandas

#### Price

In [2]:
# Define common parameters
params = {
    'time_trunc': 'hour',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Define the base URL
base_url = 'https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?'

# Get the current date and time
current_datetime = datetime.now()

# Calculate the start date as one year before the current date
start_date = current_datetime - timedelta(days=365)

# Initialize a list to store the DataFrames of prices
dfs_prices = []

# Create a loop to fetch data month by month
while start_date <= current_datetime:
    # Calculate the end date for the current month
    year = start_date.year
    month = start_date.month
    last_day_of_month = (start_date.replace(day=28) + timedelta(days=4)).replace(day=1) - timedelta(days=1)
    end_date = last_day_of_month.replace(hour=23, minute=59, second=59)

    # Add the dates to the parameters
    params['start_date'] = start_date.isoformat()
    params['end_date'] = end_date.isoformat()

    # Make the HTTP request
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the price data for the current month
    monthly_prices = data['included'][0]['attributes']['values']

    # Create a DataFrame for the current month and add it to the list
    df_month = pd.DataFrame(monthly_prices)
    dfs_prices.append(df_month)

    # Move to the next month
    start_date = last_day_of_month + timedelta(days=1)

# Concatenate all the DataFrames into one
df_all_prices = pd.concat(dfs_prices, ignore_index=True)


In [3]:
# convert column to datetime
df_all_prices['datetime'] = pd.to_datetime(df_all_prices['datetime'], utc=True)
df_all_prices['datetime'] = df_all_prices['datetime'].dt.tz_localize(None)


In [4]:
df_all_prices = df_all_prices.drop('percentage', axis=1)
df_all_prices.to_csv('../../data/df_yearly_prices.csv', index=False)

In [5]:
df_all_prices.head()

,value,datetime
0,117.23,2022-10-20 13:00:00
1,124.51,2022-10-20 14:00:00
2,132.53,2022-10-20 15:00:00
3,198.61,2022-10-20 16:00:00
4,203.70,2022-10-20 17:00:00


#### Max Monthly demand

In [6]:
# Define common parameters
params = {
    'time_trunc': 'month',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Get the current date and time
current_datetime = datetime.now()

# Round the current date to the previous night at 00:00
end_date = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)

# Calculate the start date as one year ago at 00:00
start_date = end_date - timedelta(days=365)

# Convert the dates to text strings in the "00:00" format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M')

# Define the base URL with dynamic time parameters
base_url = f'https://apidatos.ree.es/es/datos/demanda/demanda-maxima-diaria?start_date={start_date_str}&end_date={end_date_str}'

# Make the request to the API
response = requests.get(base_url, params=params)
data = response.json()

# Extract and print the data
df_max_demand = data['included'][0]['attributes']['values']


In [7]:
df_max_demand = pd.DataFrame(df_max_demand)


In [8]:
df_max_demand.head()

,value,percentage,datetime
0,642948.636,1,2022-10-20T00:00:00.000+02:00
1,686206.783,1,2022-11-30T00:00:00.000+01:00
2,694869.232,1,2022-12-13T00:00:00.000+01:00
3,778888.868,1,2023-01-24T00:00:00.000+01:00
4,761724.660,1,2023-02-07T00:00:00.000+01:00


In [9]:
df_max_demand.to_csv('../../data/df_max_demand.csv', index=False)

#### Actual demand

In [28]:
# Define common parameters
params = {
    'time_trunc': 'hour',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Define the base URL
base_url = 'https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?'

# Get the current date and time
current_datetime = datetime.now()

# Calculate the start date as one year before the current date
start_date = current_datetime - timedelta(days=365)

# Initialize a list to store the DataFrames of prices
dfs_demand = {}

# Create a loop to fetch data month by month
while start_date <= current_datetime:
    # Calculate the end date for the current month
    year = start_date.year
    month = start_date.month
    last_day_of_month = (start_date.replace(day=28) + timedelta(days=4)).replace(day=1) - timedelta(days=1)
    end_date = last_day_of_month.replace(hour=23, minute=59, second=59)

    # Add the dates to the parameters
    params['start_date'] = start_date.isoformat()
    params['end_date'] = end_date.isoformat()

    # Make the HTTP request
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract the price data for the current month
    monthly_demand_1 = data['included'][0]['attributes']['values']
    monthly_demand_2 = data['included'][1]['attributes']['values']
    monthly_demand_3 = data['included'][2]['attributes']['values']
    type_name_1 = data['included'][0]['type']
    type_name_2 = data['included'][1]['type']
    type_name_3 = data['included'][2]['type']


    # Create a DataFrame for the current month
    df_month_1 = pd.DataFrame(monthly_demand_1)
    df_month_2 = pd.DataFrame(monthly_demand_2)
    df_month_3 = pd.DataFrame(monthly_demand_3)

    # Move to the next month
    start_date = last_day_of_month + timedelta(days=1)

# Add the DataFrames to the dictionary
dfs_demand['Real'] = df_month_1
dfs_demand['Prevista'] = df_month_2
dfs_demand['Programada'] = df_month_3

   


In [29]:
dfs_demand.keys()

dict_keys(['Real', 'Prevista', 'Programada'])

In [31]:
dfs_demand['Real'].to_csv('../../data/df_real_demand.csv', index=False)
dfs_demand['Prevista'].to_csv('../../data/df_prevista_demand.csv', index=False)
dfs_demand['Programada'].to_csv('../../data/df_programada_demand.csv', index=False)

#### Structure Generation

In [93]:
# Define common parameters
params = {
    'time_trunc': 'day',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Define the base URL
base_url = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?'

# Get the current date and time
current_datetime = datetime.now()

# Calculate the start date as one year before the current date
params['start_date'] = current_datetime - timedelta(days=365)

# Calculate the end date as the current date
params['end_date'] = current_datetime

# Initialize a dictionary to store DataFrames for each 'type'
dfs_structure_by_type = {}

# Make the initial HTTP request and store the data
response = requests.get(base_url, params=params)
data = response.json()



In [102]:
for i in range(len(data['included'])):
    type_name = data['included'][i]['type']
    dfs_structure_by_type[type_name] = pd.DataFrame(data['included'][i]['attributes']['values'])

In [ ]:
consolidated_df = pd.DataFrame(columns=['values', 'percentage', 'datetime',])
for key, df in dfs_structure_by_type.items():
    # Renombra las columnas 'values' y 'datetime' para que tengan el nombre del tipo de generación
    
    # Fusiona el DataFrame actual con el consolidado en función de la columna 'datetime'
    consolidated_df = pd.merge(consolidated_df, df, on='datetime', how='outer')

# drop all the columns that contains 'percentage'
df_structure_merge = consolidated_df.loc[:,~consolidated_df.columns.str.contains('percentage')]

In [122]:
df_structure_merge.to_csv('../../data/df_structure_merge.csv', index=False)

#### Installed power

In [2]:
# Define common parameters
params = {
    'time_trunc': 'day',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Get the current date and time
current_datetime = datetime.now()

# Round the current date to the previous night at 00:00
end_date = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)

# Calculate the start date as one year ago at 00:00
start_date = end_date - timedelta(days=365)

# Convert the dates to text strings in the "00:00" format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M')

# Define the base URL with dynamic time parameters
base_url = f'https://apidatos.ree.es/es/datos/generacion/potencia-instalada?start_date={start_date_str}&end_date={end_date_str}'

# Make the request to the API
response = requests.get(base_url, params=params)
data = response.json()

# create dataframe for each type
dfs_power_by_type = {}

for i in range(len(data['included'])):
    type_name = data['included'][i]['type']

    # Initialize a DataFrame for the current type with the name as the key
    dfs_power_by_type[type_name] = pd.DataFrame()

    # Extract the 'values' data
    values = data['included'][i]['attributes']['values']

    # Create a DataFrame for the current month
    df_month = pd.DataFrame(values)

    # Concatenate the current month's data with the type's DataFrame
    dfs_power_by_type[type_name] = pd.concat([dfs_power_by_type[type_name], df_month], ignore_index=True)


In [ ]:
consolidated_df1 = pd.DataFrame(columns=['values', 'percentage', 'datetime',])
for key, df in dfs_power_by_type.items():
    # Renombra las columnas 'values' y 'datetime' para que tengan el nombre del tipo de generación
    
    # Fusiona el DataFrame actual con el consolidado en función de la columna 'datetime'
    consolidated_df1 = pd.merge(consolidated_df1, df, on='datetime', how='outer')


df_power_merge = consolidated_df1.loc[:,~consolidated_df1.columns.str.contains('percentage')]

In [8]:
df_power_merge.to_csv('../../data/df_power_merge.csv', index=False)

#### Energy exchange

In [3]:
# Define common parameters
params = {
    'time_trunc': 'day',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Get the current date and time
current_datetime = datetime.now()

# Round the current date to the previous night at 00:00
end_date = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)

# Calculate the start date as one year ago at 00:00
start_date = end_date - timedelta(days=365)

# Convert the dates to text strings in the "00:00" format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M')

# Define the base URL with dynamic time parameters
base_url = f'https://apidatos.ree.es/es/datos/intercambios/todas-fronteras-programados?start_date={start_date_str}&end_date={end_date_str}'

# Make the request to the API
response = requests.get(base_url, params=params)
data = response.json()

dfs_exchange_by_country = {}

for i in range(len(data['included']) -1):
    type_name = data['included'][i]['type']

    # Initialize a DataFrame for the current type with the name as the key
    dfs_exchange_by_country[type_name] = pd.DataFrame()

    # Extract the 'values' data
    values = data['included'][i]['attributes']['content'][2]['attributes']['values']

    # Create a DataFrame for the current month
    df_month = pd.DataFrame(values)

    # Concatenate the current month's data with the type's DataFrame
    dfs_exchange_by_country[type_name] = pd.concat([dfs_exchange_by_country[type_name], df_month], ignore_index=True)



In [ ]:
consolidates_df2 = pd.DataFrame(columns=['values', 'percentage', 'datetime',])
for key, df in dfs_exchange_by_country.items():
    # Renombra las columnas 'values' y 'datetime' para que tengan el nombre del tipo de generación
    
    # Fusiona el DataFrame actual con el consolidado en función de la columna 'datetime'
    consolidates_df2 = pd.merge(consolidates_df2, df, on='datetime', how='outer')


df_exchange_merge = consolidates_df2.loc[:,~consolidates_df2.columns.str.contains('percentage')]

In [5]:
df_exchange_merge.to_csv('../../data/df_exchange_merge.csv', index=False)

#### Energy price components

In [7]:
# Define common parameters
params = {
    'time_trunc': 'month',
    'geo_limit': 'peninsular',
    'geo_ids': '8741'
}

# Define the base URL
base_url = 'https://apidatos.ree.es/es/datos/mercados/componentes-precio-energia-cierre-desglose?'

# Get the current date and time
current_datetime = datetime.now()

# Round the current date to the previous night at 00:00
end_date = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)

# Calculate the start date as one year ago at 00:00
start_date = end_date - timedelta(days=365)

# Convert the dates to text strings in the "00:00" format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M')

params['start_date'] = start_date_str
params['end_date'] = end_date_str

# Create dictionary to store dataframes
dfs_components_daily = {}
dfs_components_intradaily = {}


# Make the request to the API
response = requests.get(base_url, params=params)
data = response.json()

# Extract and print the data
for i in range(len(data['included'])):
    type_name = data['included'][i]['type']

    # Initialize a DataFrame for the current type with the name as the key
    dfs_components_daily[type_name] = pd.DataFrame()

    # Extract the 'values' data
    values = data['included'][i]['attributes']['content'][0]['attributes']['values']


    # Create a DataFrame for the current month
    df_month = pd.DataFrame(values)

    # Concatenate the current month's data with the type's DataFrame']
    dfs_components_daily[type_name] = pd.concat([dfs_components_daily[type_name], df_month], ignore_index=True)

# Extract and print the data
for i in range(len(data['included']) - 2):
    type_name = data['included'][i]['type']

    # Initialize a DataFrame for the current type with the name as the key
    dfs_components_intradaily[type_name] = pd.DataFrame()

    # Extract the 'values' data
    values2 = data['included'][i]['attributes']['content'][1]['attributes']['values']


    # Create a DataFrame for the current month
    df_month2 = pd.DataFrame(values2)

    # Concatenate the current month's data with the type's DataFrame']
    dfs_components_intradaily[type_name] = pd.concat([dfs_components_intradaily[type_name], df_month2], ignore_index=True)



In [ ]:
consolidated_df3 = pd.DataFrame(columns=['values', 'percentage', 'datetime',])
for key, df in dfs_components_daily.items():
    # Renombra las columnas 'values' y 'datetime' para que tengan el nombre del tipo de generación
    
    # Fusiona el DataFrame actual con el consolidado en función de la columna 'datetime'
    consolidated_df3 = pd.merge(consolidated_df3, df, on='datetime', how='outer')


df_components_daily_merge = consolidated_df3.loc[:,~consolidated_df3.columns.str.contains('percentage')]

In [13]:
df_components_daily_merge.to_csv('../../data/df_components_daily_merge.csv', index=False)

In [ ]:
consolidated_df4 = pd.DataFrame(columns=['values', 'percentage', 'datetime',])
for key, df in dfs_components_intradaily.items():
    # Renombra las columnas 'values' y 'datetime' para que tengan el nombre del tipo de generación
    
    # Fusiona el DataFrame actual con el consolidado en función de la columna 'datetime'
    consolidated_df4 = pd.merge(consolidated_df4, df, on='datetime', how='outer')


df_components_intradaily_merge = consolidated_df4.loc[:,~consolidated_df4.columns.str.contains('percentage')]

In [14]:
df_components_intradaily_merge.to_csv('../../data/df_components_intradaily_merge.csv', index=False)